## Classificationn of UCI ML Breast Cancer Wisconsin (Diagnostic) dataset

This dataset is made up of data fron 569 patients, each with a diagnosis and cell nuclei features computed from images of a fine needle aspirate of a breast mass. 

---

We will use the nuclei data and a very simple multilayer perceptron to predict the whether these masses are benign or malignant. 

This notebook is adapted from [Intro to Keras with breast cancer data[ANN] by Siddarth Yadav](https://www.kaggle.com/thebrownviking20/intro-to-keras-with-breast-cancer-data-ann/notebook) ([Licensed under the Apache License, Version 2.0](http://www.apache.org/licenses/LICENSE-2.0)).

The data is available at [archive.ics.uci.edu/ml/datasets](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) and is also available via [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html).

### We begin by loading and visualising some of the data

In [1]:
from sklearn.datasets import load_breast_cancer
from IPython.display import display

x, y = load_breast_cancer(return_X_y=True, as_frame=True)

print("Nuclei measurements")
display(x)
print("Diagnosis. 0 = Malignant, 1 = Benign")
print(y)

Nuclei measurements


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


Diagnosis. 0 = Malignant, 1 = Benign
0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64


### Split the dataset into a train (80%) and test (20%) fraction

In [2]:
from sklearn.model_selection import train_test_split

test_fraction = 0.2
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_fraction
)

### Preprocess data

In [3]:
# Subtract mean and scale to unit variance
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

### Now we'll build a model to segment these images

We'll be using a very simple [Multilayer perceptron](https://en.wikipedia.org/wiki/Multilayer_perceptron)

<img src="images/mlp.png" style="width: 400px;"/>
<sub><sup>https://scikit-learn.org/stable/modules/neural_networks_supervised.html</sup></sub>

In [4]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout


def build_model():
    input = Input(
        (x_train.shape[1]),
    )
    output = Dense(16, activation="relu")(input)
    output = Dropout(0.25)(output)
    output = Dense(32, activation="relu")(output)
    output = Dropout(0.25)(output)
    output = Dense(16, activation="relu")(output)
    output = Dropout(0.25)(output)
    output = Dense(1, activation="relu")(output)
    model = Model(inputs=[input], outputs=[output])
    return model

2021-11-15 12:17:26.532275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-15 12:17:26.532305: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
model = build_model()
model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

2021-11-15 12:17:28.027116: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-15 12:17:28.027197: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-15 12:17:28.027256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-15 12:17:28.027313: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2021-11-15 12:17:28.027370: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

### Train the model

In [6]:
epochs = 50  # Number of iterations over the full dataset
batch_size = 16  # Number of samples per gradient update
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/50
29/29 [==============================] - 1s 1ms/step - loss: 3.5987 - accuracy: 0.4879
Epoch 2/50
29/29 [==============================] - 0s 1ms/step - loss: 1.4481 - accuracy: 0.6462
Epoch 3/50
29/29 [==============================] - 0s 1ms/step - loss: 2.1292 - accuracy: 0.6659
Epoch 4/50
29/29 [==============================] - 0s 1ms/step - loss: 1.4543 - accuracy: 0.7165
Epoch 5/50
29/29 [==============================] - 0s 1ms/step - loss: 1.0223 - accuracy: 0.7890
Epoch 6/50
29/29 [==============================] - 0s 1ms/step - loss: 0.7940 - accuracy: 0.8220
Epoch 7/50
29/29 [==============================] - 0s 1ms/step - loss: 0.8830 - accuracy: 0.8198
Epoch 8/50
29/29 [==============================] - 0s 1ms/step - loss: 0.7765 - accuracy: 0.8571
Epoch 9/50
29/29 [==============================] - 0s 1ms/step - loss: 0.5410 - accuracy: 0.8901
Epoch 10/50
29/29 [==============================] - 0s 1ms/step - loss: 0.6294 - accuracy: 0.8747
Epoch 11/50
29/29 [

### Apply network to (unseen) test data

In [7]:
y_pred = model.predict(x_test)
y_pred = y_pred > 0.5

import numpy as np

accuracy = (np.squeeze(y_pred) == y_test).sum() / len(y_pred) * 100

print(f"Accuracy {accuracy:.2f}%")  # Not the best metric

Accuracy 98.25%
